In [1]:
"""
Week 2: Understanding Data Quality Issues
Goal: Learn what makes data "messy" before building ML models
"""

import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries loaded")
print(f"Today: {datetime.now().strftime('%Y-%m-%d')}")

✅ Libraries loaded
Today: 2026-02-17


In [2]:
"""
Based on industry research, these are the TOP 5 problems in real datasets:

1. DUPLICATES (85% of files have this)
   - Exact duplicates: Same row appears multiple times
   - Fuzzy duplicates: "John Doe" vs "JOHN DOE" vs "John  Doe"

2. MISSING VALUES (92% of files)
   - Empty cells
   - "N/A", "NULL", "None", "-", "?"
   - Spaces that look empty

3. FORMAT INCONSISTENCIES (78% of files)
   - Dates: "01/12/2024" vs "2024-12-01" vs "Dec 1, 2024"
   - Phone: "9876543210" vs "+91-9876543210" vs "98765 43210"
   - Currency: "₹1000" vs "1,000.00" vs "1000 INR"

4. OUTLIERS (65% of files)
   - Typos: Salary ₹5,000,000 instead of ₹50,000 (extra zeros)
   - Measurement errors: Age = 150 (impossible)

5. DATA TYPE ISSUES (55% of files)
   - Numbers stored as text: "100" instead of 100
   - Dates stored as text: "2024-01-01" instead of datetime
"""

# Let's create examples of each problem
print("Understanding 5 types of data quality issues...")

Understanding 5 types of data quality issues...


In [3]:
"""
Instead of downloading 100 files, let's CREATE messy data.
Why? Because we need to UNDERSTAND problems before detecting them.
"""

# Example 1: DUPLICATES
print("=" * 50)
print("PROBLEM 1: DUPLICATES")
print("=" * 50)

messy_data_duplicates = pd.DataFrame({
    'Name': ['John Doe', 'JOHN DOE', 'John Doe', 'Jane Smith', 'Jane Smith'],
    'Email': ['john@email.com', 'john@email.com', 'john@email.com', 
              'jane@email.com', 'jane@email.com'],
    'Phone': ['9876543210', '9876543210', '9876543210', 
              '9123456789', '9123456789']
})

print("\nMessy DataFrame:")
print(messy_data_duplicates)
print(f"\nRows: {len(messy_data_duplicates)}")
print(f"Unique people: Only 2 (John and Jane)")
print(f"Duplicates: 3 extra rows wasting space")

# How to detect?
duplicates = messy_data_duplicates.duplicated()
print(f"\nDuplicate rows: {duplicates.sum()}")
print("\nWhich rows are duplicates?")
print(messy_data_duplicates[duplicates])

PROBLEM 1: DUPLICATES

Messy DataFrame:
         Name           Email       Phone
0    John Doe  john@email.com  9876543210
1    JOHN DOE  john@email.com  9876543210
2    John Doe  john@email.com  9876543210
3  Jane Smith  jane@email.com  9123456789
4  Jane Smith  jane@email.com  9123456789

Rows: 5
Unique people: Only 2 (John and Jane)
Duplicates: 3 extra rows wasting space

Duplicate rows: 2

Which rows are duplicates?
         Name           Email       Phone
2    John Doe  john@email.com  9876543210
4  Jane Smith  jane@email.com  9123456789


In [4]:
print("=" * 50)
print("PROBLEM 2: MISSING VALUES")
print("=" * 50)

messy_data_missing = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'Email': ['alice@email.com', None, 'charlie@email.com', '', 'eve@email.com'],
    'Age': [25, 30, None, 35, 28],
    'Salary': [50000, None, 60000, 55000, None]
})

print("\nMessy DataFrame:")
print(messy_data_missing)

# Analyze missing data
print("\n--- Missing Value Analysis ---")
print(f"Total cells: {messy_data_missing.size}")
print(f"Missing cells: {messy_data_missing.isna().sum().sum()}")
print(f"Missing percentage: {(messy_data_missing.isna().sum().sum() / messy_data_missing.size) * 100:.1f}%")

print("\nMissing per column:")
for col in messy_data_missing.columns:
    missing = messy_data_missing[col].isna().sum()
    print(f"  {col}: {missing} missing ({missing/len(messy_data_missing)*100:.0f}%)")

PROBLEM 2: MISSING VALUES

Messy DataFrame:
      Name              Email   Age   Salary
0    Alice    alice@email.com  25.0  50000.0
1      Bob                NaN  30.0      NaN
2  Charlie  charlie@email.com   NaN  60000.0
3    David                     35.0  55000.0
4      Eve      eve@email.com  28.0      NaN

--- Missing Value Analysis ---
Total cells: 20
Missing cells: 4
Missing percentage: 20.0%

Missing per column:
  Name: 0 missing (0%)
  Email: 1 missing (20%)
  Age: 1 missing (20%)
  Salary: 2 missing (40%)


In [5]:
print("=" * 50)
print("PROBLEM 3: FORMAT INCONSISTENCIES")
print("=" * 50)

messy_data_format = pd.DataFrame({
    'Name': ['John', 'Jane', 'Bob', 'Alice'],
    'Phone': [
        '9876543210',           # Just numbers
        '+91-9876543210',       # With country code
        '98765 43210',          # With space
        '(+91) 987-654-3210'    # Full format
    ],
    'Date': [
        '01/12/2024',           # DD/MM/YYYY
        '2024-12-01',           # YYYY-MM-DD
        'Dec 1, 2024',          # Text format
        '01-Dec-24'             # Short format
    ]
})

print("\nMessy DataFrame:")
print(messy_data_format)

print("\n--- Problem Analysis ---")
print("Phone numbers have 4 different formats!")
print("Dates have 4 different formats!")
print("This makes comparison/sorting impossible.")

PROBLEM 3: FORMAT INCONSISTENCIES

Messy DataFrame:
    Name               Phone         Date
0   John          9876543210   01/12/2024
1   Jane      +91-9876543210   2024-12-01
2    Bob         98765 43210  Dec 1, 2024
3  Alice  (+91) 987-654-3210    01-Dec-24

--- Problem Analysis ---
Phone numbers have 4 different formats!
Dates have 4 different formats!
This makes comparison/sorting impossible.


In [6]:
print("=" * 50)
print("PROBLEM 4: OUTLIERS (Typos)")
print("=" * 50)

messy_data_outliers = pd.DataFrame({
    'Name': ['John', 'Jane', 'Bob', 'Alice', 'Charlie'],
    'Salary': [50000, 55000, 5000000, 52000, 250],  # Row 2: extra zeros, Row 4: missing zeros
    'Age': [25, 30, 28, 150, 27]  # Row 3: impossible age
})

print("\nMessy DataFrame:")
print(messy_data_outliers)

# Statistical analysis
print("\n--- Salary Analysis ---")
print(f"Mean: ₹{messy_data_outliers['Salary'].mean():,.0f}")
print(f"Median: ₹{messy_data_outliers['Salary'].median():,.0f}")
print(f"Problem: Mean is way higher than median (outlier effect)")

print("\n--- Age Analysis ---")
print(f"Ages: {messy_data_outliers['Age'].tolist()}")
print(f"Max age: {messy_data_outliers['Age'].max()} ← Clearly wrong!")

PROBLEM 4: OUTLIERS (Typos)

Messy DataFrame:
      Name   Salary  Age
0     John    50000   25
1     Jane    55000   30
2      Bob  5000000   28
3    Alice    52000  150
4  Charlie      250   27

--- Salary Analysis ---
Mean: ₹1,031,450
Median: ₹52,000
Problem: Mean is way higher than median (outlier effect)

--- Age Analysis ---
Ages: [25, 30, 28, 150, 27]
Max age: 150 ← Clearly wrong!


In [7]:
print("=" * 50)
print("PROBLEM 5: DATA TYPE ISSUES")
print("=" * 50)

messy_data_types = pd.DataFrame({
    'Product': ['Laptop', 'Phone', 'Tablet'],
    'Quantity': ['100', '50', '75'],      # Should be numbers!
    'Price': ['50000', '30000', '25000'], # Should be numbers!
    'In_Stock': ['Yes', 'No', 'Yes']      # Should be True/False!
})

print("\nMessy DataFrame:")
print(messy_data_types)
print("\nData types:")
print(messy_data_types.dtypes)

print("\n--- Problem ---")
print("Quantity and Price are 'object' (text) instead of numbers")
print("Can't calculate sum/average!")

# Try to sum
try:
    total = messy_data_types['Quantity'].sum()
    print(f"\nSum of Quantity: {total}")
    print("^ This concatenates strings instead of adding numbers!")
except:
    print("Can't sum text!")

PROBLEM 5: DATA TYPE ISSUES

Messy DataFrame:
  Product Quantity  Price In_Stock
0  Laptop      100  50000      Yes
1   Phone       50  30000       No
2  Tablet       75  25000      Yes

Data types:
Product     str
Quantity    str
Price       str
In_Stock    str
dtype: object

--- Problem ---
Quantity and Price are 'object' (text) instead of numbers
Can't calculate sum/average!

Sum of Quantity: 1005075
^ This concatenates strings instead of adding numbers!


In [8]:
"""
KEY LEARNINGS:
--------------

1. Data quality issues are PATTERNS, not random
2. Same problems appear in 80%+ of datasets
3. Each problem needs different detection method:
   - Duplicates: Compare rows
   - Missing: Check for None/NaN/empty
   - Format: Regex pattern matching
   - Outliers: Statistical analysis (mean/median/std)
   - Types: Check dtype vs expected type

4. Manual inspection is important BEFORE building ML

NEXT STEPS:
-----------
- Find 10-15 REAL messy files
- Manually label problems in each
- Document patterns we see
- This becomes our training data
"""

print("\n✅ Week 2 Day 1-2 Complete!")
print("\nNext: Find real messy data from Kaggle/government sites")


✅ Week 2 Day 1-2 Complete!

Next: Find real messy data from Kaggle/government sites
